In [48]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
import math
import time
from datetime import datetime
import pickle
from pathlib import Path
import os
import requests
from bs4 import BeautifulSoup
import traceback
import sys
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [15]:
driver = webdriver.Chrome()
book_list_urls = [
    'https://reader.litres.ru/vybor/small',
    'https://reader.litres.ru/vybor/big',
    'https://reader.litres.ru/vybor/modern',
    'https://reader.litres.ru/vybor/samizdat'
]

book_links = []

with open('SECRET_LINK.txt') as f:
    login_link = f.read()
driver.get(login_link)

for book_list_url in book_list_urls:
    book_list_url += '?sort=name-book' + '&per-page=100&page='
    driver.get(book_list_url)
    books_n = driver.find_element(By.CLASS_NAME, "summary").text
    books_n = books_n[books_n.find('из') + 3:-1].replace(' ', '')
    books_n = int(books_n)
    print(f'{books_n}')
    for page in tqdm(range(1, math.ceil(books_n / 100) + 1)):
        page_url = book_list_url + str(page)
        driver.get(page_url)
        for book_button in driver.find_elements(By.PARTIAL_LINK_TEXT, 'ОТКРЫТЬ НА LITRES'):
            book_links.append(book_button.get_attribute("href"))
        time.sleep(0.5)
driver.close()

1276


100%|██████████| 13/13 [00:23<00:00,  1.83s/it]


949


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


1168


100%|██████████| 12/12 [00:23<00:00,  1.96s/it]


794


100%|██████████| 8/8 [00:15<00:00,  1.91s/it]


# Pickle the links list

In [53]:
filename = 'books_list_' + datetime.now().strftime("%m-%d-%Y_%H-%M") + '.pkl'
output_folder = Path('output')
output_folder.mkdir(exist_ok=True)
with open(output_folder / filename, 'wb') as f:
    pickle.dump(book_links, f)

# Load from pickle

In [6]:
output_folder = Path('output')
paths = sorted(Path(output_folder).iterdir(), key=os.path.getmtime)
idx = -1
while 'books_list' not in paths[idx].name:
    idx -= 1
with open(paths[idx], 'rb') as f:
    book_links = pickle.load(f)
print(len(book_links))

4187


In [115]:
books_db = []

for i, book_link in (pbar := tqdm(enumerate(book_links))):
    pbar.set_description(book_link)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(book_link, headers=headers)
    if r.status_code != 200:
        if r.status_code == 400:
            continue
        else:
            print(f'status code is {r.status_code}! Full html:\n{r.text}')
            break
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        title = soup.find('div', {'class': 'biblio_book_name biblio-book__title-block'})
        title = title.h1.text
        author = soup.find('div', {'class': 'biblio_book_author'})
        author = author.a.span.text
    except Exception as e:
        print(f"Couldn't find a book name or an author! {page_url=}")
        print(e)
        print(traceback.format_exc())
        break  # something is definitely goes wrong
    try:
        soup.find(itemprop="aggregateRating")
        rating_div = soup.find(itemprop="aggregateRating")
        mean = rating_div.findChild("div", {'class': "rating-number"}).text
        mean = float(mean.replace(',', '.'))
        n_votes = rating_div.findChild("div", {'class': "votes-count"}).text
        n_votes = int(n_votes)
    except AttributeError as e:
        mean, n_votes = 0, 0
    pages = 0
    published = '???'
    try:
        info_block = soup.find('ul', {'class': 'biblio_book_info_detailed_left'})
        info_lis = info_block.findAll('li')
        for info in info_lis:
            if 'Дата выхода на ЛитРес' in info.text:
                published = info.text[len('Дата выхода на ЛитРес: '):]
            if 'Объем: ' in info.text:
                pages = info.text
                pages = pages[len('Объем: '):pages.find('стр.')]
                pages = int(pages)
    except AttributeError as e:
        pages = 0
    book = {
        'author': author,
        'title': title,
        'pages': pages,
        'mean_rating': mean,
        'n_votes': n_votes,
        'link': book_link,
        'published': published
    }
    books_db.append(book)
    if (i+1) % 10 == 0:
        time.sleep(5)
filename = 'books_db_' + datetime.now().strftime("%m-%d-%Y_%H-%M") + '.pkl'
output_folder = Path('output')
output_folder.mkdir(exist_ok=True)
with open(output_folder / filename, 'wb') as f:
    pickle.dump(books_db, f)

https://www.litres.ru/leonid-andreev/v-saburove/: : 34it [01:34,  2.79s/it]                                        


KeyboardInterrupt: 

In [116]:
for book in books_db:
    print(book)

{'author': 'Саша Чёрный', 'title': '«Лебединая прохлада»', 'pages': 13, 'mean_rating': 4.8, 'n_votes': 4, 'link': 'https://www.litres.ru/sasha-chernyy/lebedinaya-prohlada/', 'published': '04 марта 2013'}
{'author': 'Лазарь Осипович Кармен', 'title': '<Под рождество>', 'pages': 14, 'mean_rating': 3.8, 'n_votes': 7, 'link': 'https://www.litres.ru/lazar-karmen/pod-rozhdestvo/', 'published': '27 ноября 2016'}
{'author': 'Владимир Гиляровский', 'title': 'А. И. Южин', 'pages': 6, 'mean_rating': 4.3, 'n_votes': 4, 'link': 'https://www.litres.ru/vladimir-gilyarovskiy/a-i-uzhin/', 'published': '16 ноября 2008'}
{'author': 'Владимир Гиляровский', 'title': 'Актер Вольский', 'pages': 14, 'mean_rating': 4.7, 'n_votes': 3, 'link': 'https://www.litres.ru/vladimir-gilyarovskiy/akter-volskiy/', 'published': '16 ноября 2008'}
{'author': 'Александр Пушкин', 'title': 'Анджело', 'pages': 14, 'mean_rating': 4.6, 'n_votes': 22, 'link': 'https://www.litres.ru/aleksandr-pushkin/andzhelo/', 'published': '22 дек

In [118]:
sorted(books_db, key=lambda book: -book['n_votes'])

[{'author': 'Александр Островский',
  'title': 'Бесприданница',
  'pages': 80,
  'mean_rating': 4.8,
  'n_votes': 1944,
  'link': 'https://www.litres.ru/aleksandr-ostrovskiy/bespridannica/',
  'published': '14 ноября 2008'},
 {'author': 'Михаил Лермонтов',
  'title': 'Ашик-Кериб',
  'pages': 10,
  'mean_rating': 4.6,
  'n_votes': 613,
  'link': 'https://www.litres.ru/mihail-lermontov/ashik-kerib-175941/',
  'published': '17 декабря 2008'},
 {'author': 'Александр Пушкин',
  'title': 'Арап Петра Великого',
  'pages': 39,
  'mean_rating': 4.5,
  'n_votes': 155,
  'link': 'https://www.litres.ru/aleksandr-pushkin/arap-petra-velikogo/',
  'published': '25 декабря 2008'},
 {'author': 'Надежда Тэффи',
  'title': 'Брошечка',
  'pages': 5,
  'mean_rating': 4.7,
  'n_votes': 55,
  'link': 'https://www.litres.ru/nadezhda-teffi/broshechka/',
  'published': '14 ноября 2008'},
 {'author': 'Антон Чехов',
  'title': 'Брак по расчету',
  'pages': 5,
  'mean_rating': 4.7,
  'n_votes': 34,
  'link': 'http

Возрастное ограничение: 12+
			
Дата выхода на ЛитРес: 16 октября 2009


In [111]:
published

'16 октября 2009'

In [119]:
first = './output/books_db_14_06-26-2022_13-04.pkl'
second = './output/books_db_7_06-26-2022_13-06.pkl'

with open(first, 'rb') as f:
    first = pickle.load(f)
with open(second, 'rb') as f:
    second = pickle.load(f)

print("First: ")
for x in first:
    print(x['title'], end=" ")
print()
print("Second: ")
for x in second:
    print(x['title'], end=" ")
print()

First: 
«Лебединая прохлада» <Под рождество> А. И. Южин Актер Вольский Анджело Антей Аптечка Арап Петра Великого Армейский спотыкач Артур Шопенгауэр. Его жизнь и научная деятельность Ашик-Кериб Аэродром Бабушка Акулина Балабурда 
Second: 
Бабушка Акулина Балабурда Башня в плюще Безгласное королевство Белый огонь Белый охотник Бен-Товит 
